In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import rasterio
from rasterio.windows import Window
from tqdm.notebook import tqdm, trange

In [2]:
# IRRAD = srad, TMIN = tmin, TMAX = tmax, VAP = vp, WIND = 3, RAIN = prcp, SNOWDEPTH = NaN 

In [3]:
# Select a window without missing data in the center of the tile:
w = Window(22, 31, 200, 200)
dm_vars = ['dayl', 'srad', 'tmin', 'tmax', 'vp', 'prcp']

In [4]:
daymet = []
for y in trange(2009, 2020):
    yl = []
    for v in dm_vars:
        with rasterio.open(f'../../Datasets/DAYMET/{v}_{y}_9584.nc') as rst:
            yl.append(rst.read(window = w, out_shape = (40,40)))
    daymet.append(np.stack(yl, 1))
daymet = np.stack(daymet)

In [5]:
#Years,Days,Variables,H,W
daymet.shape

(11, 365, 6, 40, 40)

In [6]:
#Time,Variables,H,W
daymetf = np.concatenate(daymet)

# Fix the radiation values and the VP units:
daymetf[:,1] = 1e-03 * daymetf[:,0] * daymetf[:,1]
daymetf[:,4] = 1e-03 * daymetf[:,4]

# Drop the dayl variable:
daymetf = daymetf[:,1:]

daymetf.shape

(4015, 5, 40, 40)

In [7]:
# Get the years that were discarded in leap years:
days = []
for y in range(2009, 2020):
    ydays = pd.to_datetime(f'{y}0101', format= '%Y%m%d')
    ydays = ydays + pd.to_timedelta(np.arange(365), unit='d')
    days.append(ydays)
days = np.concatenate(days)
td = np.diff(days)
lidx = np.where(~(td == td[0]))[0]
lidx

array([1459, 2919])

In [8]:
# Duplicate the data from 12/30 to fill the missign 12/31:
daymetf = np.insert(daymetf, lidx, daymetf[lidx], axis = 0)

In [9]:
# Create the sequential dates:
ydays = pd.to_datetime('20090101', format= '%Y%m%d')
ydays = ydays + pd.to_timedelta(np.arange(len(daymetf)), unit='d')
ydays = ydays.strftime('%Y%m%d').values.astype('int32')
ydays = np.zeros_like(daymetf[:,[0]]) + ydays[:,None,None,None]

In [10]:
# Create dummy values to fill the missing variables
wind = 3 * np.ones_like(daymetf[:,[0]])
snow = np.zeros_like(daymetf[:,[0]])
snow[:] = np.nan

In [11]:
# Organize the variables according to the order fo the CSV:
daymetf = np.concatenate([ydays, daymetf[:,:4], wind, daymetf[:,[4]], snow], axis = 1)

In [12]:
svars = ['DAY', 'IRRAD', 'TMIN', 'TMAX', 'VAP', 'WIND', 'RAIN', 'SNOWDEPTH']
daymetf.shape

(4017, 8, 40, 40)

In [13]:
hh = ['## Site Characteristics\n',
 "Country     = 'Mexico'\n",
 "Station     = '9584'\n",
 "Description = 'CHIAPAS'\n",
 "Source      = 'DAYMET'\n",
 "Contact     = 'Rodrigo'\n",
 'Longitude   = -93\n',
 'Latitude    = 17\n',
 'Elevation   = 500\n',
 'AngstromA   = 0.3\n',
 'AngstromB   = 0.5\n',
 'HasSunshine = False\n',
 '## Daily weather observations\n',
 'DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH\n']

In [14]:
pxy = np.stack(np.meshgrid(np.arange(daymetf.shape[-2]), np.arange(daymetf.shape[-1])), -1).reshape(-1, 2)

In [15]:
px = 0
py = 0
for px, py in tqdm(pxy):
    dff = pd.DataFrame(daymetf[:,:,px,py,], columns = svars)
    dff.DAY = dff.DAY.astype('int').astype('str')
    dff.VAP = np.clip(dff.VAP, 0.06, 199.3)

    save_file = f'/home/rodrigo7/Apsim_test/MASAGRO/DAYMET_TILE/DAYMET_9584_{px:02d}_{py:02d}.csv'
    with open(save_file, 'w') as sf:
        for l in hh:
            sf.writelines(l)
    dff.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)